# ICH Parameter Sensitivity Analysis

**Objective**: Determine whether the ICH model produces Λ-like behavior ($w_{\text{eff}} \approx -1$) across broad parameter ranges, or requires fine-tuning.

## Key Questions
1. How sensitive is $w_{\text{eff}}$ to influx parameters?
2. How sensitive is $w_{\text{eff}}$ to black hole parameters?
3. How sensitive is $w_{\text{eff}}$ to the imbalance coupling?

## Method
- Sweep each parameter independently while holding others at baseline
- Record final $w_{\text{eff}}$, $\chi^2$, and circulation statistics
- Identify stable regions where $-1.5 < w_{\text{eff}} < -0.7$

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
from scipy.interpolate import interp1d
from scipy.stats import chi2
import pandas as pd
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

## Baseline Parameters

These are the reference values from the main simulation. Sweeps will vary one parameter at a time.

In [ ]:
BASELINE = {
    # Simulation control
    'N_STEPS': 20000,
    'UNIVERSE_SIZE_0': 80.0,
    'ACCEPT_PROB': 0.08,
    'A_INITIAL': 0.25,
    
    # Influx parameters
    'INFLUX_PEAK': 0.25,
    'INFLUX_FLOOR': 0.02,
    'ENTROPY_SENSITIVITY_K': 3.0,
    'ENTROPY_DRIFT': 0.0006,
    'CLUSTER_PULL': 0.06,
    
    # Black hole parameters
    'BH_ABSORB_RADIUS': 6,
    'BH_SHRINK_PER_PROCESS': 0.5,
    'BH_HAWKING_LEAK': 0.003,
    'BH_MIN_MASS': 2.0,
    'BH_FORM_WINDOW': 15,
    'BH_FORM_DENSITY_TH': 0.5,
    'BH_FORM_ENTROPY_MAX': 0.6,
    'BH_FORM_PROB': 0.03,
    'BH_INITIAL_MASS_NEW': 30,
    'BH_MERGE_COMOVING_DIST': 12.0,
    'BH_MERGE_PROB': 0.35,
    'BH_MERGE_EFFICIENCY': 0.92,
    
    # Cosmology
    'H0': 0.00005,
    'DT': 1.0,
    'OMEGA_M0': 0.30,
    'IMBALANCE_COUPLING': 0.002,
    'DILUTION_POWER': 3.2,
    'W_EFF_EMA_ALPHA': 0.08,
    
    # Distance modulus
    'REF_Z_POINTS': [0.44, 1.0],
    'REF_MU_VALUES': [42.8, 44.3],
    'REF_SIGMA': [0.18, 0.20],
    'Z_INTEGRATION_POINTS': 100,
    'Z_LOW_NORMALIZATION': 0.05,
    'TARGET_MU_LOWZ': 36.5,
}

print("Baseline parameters loaded.")

## Simulation Function

Encapsulated version of the main simulation for batch runs.

In [ ]:
def run_ich_simulation(params, seed=None, verbose=False):
    """
    Run ICH simulation with given parameters.
    Returns summary statistics.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    # Unpack parameters
    N_STEPS = params['N_STEPS']
    UNIVERSE_SIZE_0 = params['UNIVERSE_SIZE_0']
    ACCEPT_PROB = params['ACCEPT_PROB']
    A_INITIAL = params['A_INITIAL']
    INFLUX_PEAK = params['INFLUX_PEAK']
    INFLUX_FLOOR = params['INFLUX_FLOOR']
    ENTROPY_SENSITIVITY_K = params['ENTROPY_SENSITIVITY_K']
    ENTROPY_DRIFT = params['ENTROPY_DRIFT']
    CLUSTER_PULL = params['CLUSTER_PULL']
    BH_ABSORB_RADIUS = params['BH_ABSORB_RADIUS']
    BH_SHRINK_PER_PROCESS = params['BH_SHRINK_PER_PROCESS']
    BH_HAWKING_LEAK = params['BH_HAWKING_LEAK']
    BH_MIN_MASS = params['BH_MIN_MASS']
    BH_FORM_WINDOW = params['BH_FORM_WINDOW']
    BH_FORM_DENSITY_TH = params['BH_FORM_DENSITY_TH']
    BH_FORM_ENTROPY_MAX = params['BH_FORM_ENTROPY_MAX']
    BH_FORM_PROB = params['BH_FORM_PROB']
    BH_INITIAL_MASS_NEW = params['BH_INITIAL_MASS_NEW']
    BH_MERGE_COMOVING_DIST = params['BH_MERGE_COMOVING_DIST']
    BH_MERGE_PROB = params['BH_MERGE_PROB']
    BH_MERGE_EFFICIENCY = params['BH_MERGE_EFFICIENCY']
    H0 = params['H0']
    DT = params['DT']
    OMEGA_M0 = params['OMEGA_M0']
    IMBALANCE_COUPLING = params['IMBALANCE_COUPLING']
    DILUTION_POWER = params['DILUTION_POWER']
    W_EFF_EMA_ALPHA = params['W_EFF_EMA_ALPHA']
    
    # State
    particles = []
    black_holes = []
    a = A_INITIAL
    H = H0
    total_instantiated = 0
    total_recycled = 0
    total_bh_formed = 0
    cumulative_imbalance = 0.0
    da_dt_history = deque(maxlen=5)
    a_history = deque(maxlen=5)
    smoothed_d2a_dt2 = 0.0
    smoothed_w_eff = -1.0
    
    w_eff_samples = []
    
    def physical_volume():
        return UNIVERSE_SIZE_0 * a
    
    def compute_avg_entropy():
        if not particles:
            return 0.0
        return np.mean([p["entropy"] for p in particles])
    
    def get_influx_rate(avg_entropy):
        suppression = 1 + ENTROPY_SENSITIVITY_K * avg_entropy
        return INFLUX_FLOOR + (INFLUX_PEAK - INFLUX_FLOOR) / suppression
    
    def add_new_particles(avg_entropy):
        nonlocal total_instantiated
        rate = get_influx_rate(avg_entropy)
        n_attempts = int(physical_volume() * rate)
        added = 0
        for _ in range(n_attempts):
            if random.random() < ACCEPT_PROB:
                pos = random.uniform(0, UNIVERSE_SIZE_0)
                particles.append({
                    "pos": pos,
                    "entropy": 0.04 + random.random() * 0.22,
                    "age": 0
                })
                added += 1
                total_instantiated += 1
        return added, rate
    
    def evolve_particles():
        if not particles:
            return
        bh_positions = [bh["pos"] for bh in black_holes if bh["active"]]
        for p in particles:
            p["entropy"] += ENTROPY_DRIFT
            p["age"] += 1
            if bh_positions:
                min_dist = float('inf')
                nearest_bh_pos = None
                for bh_pos in bh_positions:
                    dist = abs(p["pos"] - bh_pos)
                    dist = min(dist, UNIVERSE_SIZE_0 - dist)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_bh_pos = bh_pos
                if nearest_bh_pos is not None and min_dist > 0:
                    direction = 1 if nearest_bh_pos > p["pos"] else -1
                    if abs(p["pos"] - nearest_bh_pos) > UNIVERSE_SIZE_0 / 2:
                        direction *= -1
                    pull = CLUSTER_PULL / (1 + min_dist * 0.1)
                    p["pos"] += direction * pull
                    p["pos"] = p["pos"] % UNIVERSE_SIZE_0
    
    def absorb_and_evaporate():
        nonlocal total_recycled
        recycled_this_step = 0
        for bh in black_holes:
            if not bh["active"]:
                continue
            to_remove = []
            for i, p in enumerate(particles):
                dist = abs(p["pos"] - bh["pos"])
                dist = min(dist, UNIVERSE_SIZE_0 - dist)
                if dist < BH_ABSORB_RADIUS:
                    to_remove.append(i)
                    bh["mass"] += 1
            for i in reversed(to_remove):
                particles.pop(i)
                recycled_this_step += 1
                total_recycled += 1
            if to_remove:
                bh["mass"] -= BH_SHRINK_PER_PROCESS
            bh["mass"] -= BH_HAWKING_LEAK
            if bh["mass"] < BH_MIN_MASS:
                bh["active"] = False
        return recycled_this_step
    
    def check_and_merge_black_holes():
        active_bhs = [bh for bh in black_holes if bh["active"]]
        if len(active_bhs) < 2:
            return
        merged_indices = set()
        for i in range(len(active_bhs)):
            if i in merged_indices:
                continue
            for j in range(i + 1, len(active_bhs)):
                if j in merged_indices:
                    continue
                bh1, bh2 = active_bhs[i], active_bhs[j]
                dist = abs(bh1["pos"] - bh2["pos"])
                dist = min(dist, UNIVERSE_SIZE_0 - dist)
                if dist < BH_MERGE_COMOVING_DIST and random.random() < BH_MERGE_PROB:
                    if bh1["mass"] >= bh2["mass"]:
                        bh1["mass"] += bh2["mass"] * BH_MERGE_EFFICIENCY
                        bh2["active"] = False
                    else:
                        bh2["mass"] += bh1["mass"] * BH_MERGE_EFFICIENCY
                        bh1["active"] = False
                    merged_indices.add(j)
    
    def try_form_black_hole():
        nonlocal total_bh_formed
        if not particles:
            return
        center = random.uniform(0, UNIVERSE_SIZE_0)
        local_particles = []
        for p in particles:
            dist = abs(p["pos"] - center)
            dist = min(dist, UNIVERSE_SIZE_0 - dist)
            if dist < BH_FORM_WINDOW:
                local_particles.append(p)
        if not local_particles:
            return
        local_density = len(local_particles) / (2 * BH_FORM_WINDOW)
        if local_density < BH_FORM_DENSITY_TH:
            return
        local_entropy = np.mean([p["entropy"] for p in local_particles])
        if local_entropy > BH_FORM_ENTROPY_MAX:
            return
        if random.random() < BH_FORM_PROB:
            black_holes.append({
                "pos": center,
                "mass": BH_INITIAL_MASS_NEW,
                "active": True
            })
            total_bh_formed += 1
    
    def update_cosmology(added, recycled, avg_entropy):
        nonlocal a, H, cumulative_imbalance, smoothed_d2a_dt2, smoothed_w_eff
        imbalance = added - recycled
        cumulative_imbalance += imbalance
        rho_L_eff = IMBALANCE_COUPLING * cumulative_imbalance / (a ** DILUTION_POWER) if a > 0 else 0
        rho_m = OMEGA_M0 / (a ** 3) if a > 0 else OMEGA_M0
        rho_total = rho_m + max(rho_L_eff, 0)
        H = H0 * np.sqrt(rho_total / (OMEGA_M0 + 0.001)) if rho_total > 0 else H0 * 0.1
        da_dt = H * a
        da_dt_history.append(da_dt)
        a_history.append(a)
        a += H * a * DT
        w_eff = smoothed_w_eff
        if len(da_dt_history) >= 3:
            d2a_dt2 = (da_dt_history[-1] - 2*da_dt_history[-2] + da_dt_history[-3]) / (DT**2)
            smoothed_d2a_dt2 = W_EFF_EMA_ALPHA * d2a_dt2 + (1 - W_EFF_EMA_ALPHA) * smoothed_d2a_dt2
            if rho_L_eff > 1e-10 and a > 0:
                w_raw = -1 - (2 * smoothed_d2a_dt2 * a) / (3 * H**2 * a**2 + 1e-10)
                w_eff = max(-2.5, min(-0.3, w_raw))
                smoothed_w_eff = W_EFF_EMA_ALPHA * w_eff + (1 - W_EFF_EMA_ALPHA) * smoothed_w_eff
        return rho_L_eff, imbalance, smoothed_w_eff
    
    # Main loop
    for step in range(N_STEPS):
        avg_entropy = compute_avg_entropy()
        added, _ = add_new_particles(avg_entropy)
        evolve_particles()
        recycled = absorb_and_evaporate()
        check_and_merge_black_holes()
        try_form_black_hole()
        rho_L_eff, imb, w_proxy = update_cosmology(added, recycled, avg_entropy)
        
        # Sample w_eff in second half of simulation
        if step > N_STEPS // 2 and step % 100 == 0:
            w_eff_samples.append(w_proxy)
        
        if a >= 1.0:
            break
    
    # Compute summary
    final_w_eff = np.mean(w_eff_samples) if w_eff_samples else smoothed_w_eff
    w_eff_std = np.std(w_eff_samples) if len(w_eff_samples) > 1 else 0.0
    
    return {
        'final_a': a,
        'final_w_eff': final_w_eff,
        'w_eff_std': w_eff_std,
        'total_instantiated': total_instantiated,
        'total_recycled': total_recycled,
        'net_imbalance': total_instantiated - total_recycled,
        'total_bh_formed': total_bh_formed,
        'final_particles': len(particles),
        'final_bh': sum(bh["active"] for bh in black_holes),
        'reached_present': a >= 1.0
    }

## Test Baseline

In [ ]:
print("Testing baseline parameters...")
result = run_ich_simulation(BASELINE, seed=42)
print(f"Final a: {result['final_a']:.4f}")
print(f"Final w_eff: {result['final_w_eff']:.3f} ± {result['w_eff_std']:.3f}")
print(f"Circulation: {result['total_instantiated']} in, {result['total_recycled']} out")
print(f"Reached present: {result['reached_present']}")

## Parameter Sweeps

### Sweep 1: Influx Peak Rate

In [ ]:
influx_peak_values = np.linspace(0.10, 0.50, 9)
influx_peak_results = []

print("Sweeping INFLUX_PEAK...")
for val in tqdm(influx_peak_values):
    params = BASELINE.copy()
    params['INFLUX_PEAK'] = val
    result = run_ich_simulation(params, seed=42)
    result['param_value'] = val
    influx_peak_results.append(result)

df_influx_peak = pd.DataFrame(influx_peak_results)
print(df_influx_peak[['param_value', 'final_w_eff', 'w_eff_std', 'net_imbalance', 'reached_present']])

### Sweep 2: Entropy Sensitivity

In [ ]:
entropy_k_values = np.linspace(1.0, 6.0, 9)
entropy_k_results = []

print("Sweeping ENTROPY_SENSITIVITY_K...")
for val in tqdm(entropy_k_values):
    params = BASELINE.copy()
    params['ENTROPY_SENSITIVITY_K'] = val
    result = run_ich_simulation(params, seed=42)
    result['param_value'] = val
    entropy_k_results.append(result)

df_entropy_k = pd.DataFrame(entropy_k_results)
print(df_entropy_k[['param_value', 'final_w_eff', 'w_eff_std', 'net_imbalance', 'reached_present']])

### Sweep 3: Imbalance Coupling

In [ ]:
coupling_values = np.logspace(-4, -1, 9)  # 0.0001 to 0.1
coupling_results = []

print("Sweeping IMBALANCE_COUPLING...")
for val in tqdm(coupling_values):
    params = BASELINE.copy()
    params['IMBALANCE_COUPLING'] = val
    result = run_ich_simulation(params, seed=42)
    result['param_value'] = val
    coupling_results.append(result)

df_coupling = pd.DataFrame(coupling_results)
print(df_coupling[['param_value', 'final_w_eff', 'w_eff_std', 'net_imbalance', 'reached_present']])

### Sweep 4: Black Hole Absorption Radius

In [ ]:
bh_radius_values = np.linspace(2, 12, 9)
bh_radius_results = []

print("Sweeping BH_ABSORB_RADIUS...")
for val in tqdm(bh_radius_values):
    params = BASELINE.copy()
    params['BH_ABSORB_RADIUS'] = val
    result = run_ich_simulation(params, seed=42)
    result['param_value'] = val
    bh_radius_results.append(result)

df_bh_radius = pd.DataFrame(bh_radius_results)
print(df_bh_radius[['param_value', 'final_w_eff', 'w_eff_std', 'net_imbalance', 'reached_present']])

### Sweep 5: Dilution Power

In [ ]:
dilution_values = np.linspace(2.5, 4.0, 9)
dilution_results = []

print("Sweeping DILUTION_POWER...")
for val in tqdm(dilution_values):
    params = BASELINE.copy()
    params['DILUTION_POWER'] = val
    result = run_ich_simulation(params, seed=42)
    result['param_value'] = val
    dilution_results.append(result)

df_dilution = pd.DataFrame(dilution_results)
print(df_dilution[['param_value', 'final_w_eff', 'w_eff_std', 'net_imbalance', 'reached_present']])

## Visualization

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(14, 9))

# Common styling
lambda_band = (-1.1, -0.9)  # Λ-like region

# 1. Influx Peak
ax = axs[0, 0]
ax.errorbar(df_influx_peak['param_value'], df_influx_peak['final_w_eff'], 
            yerr=df_influx_peak['w_eff_std'], fmt='o-', capsize=3)
ax.axhspan(lambda_band[0], lambda_band[1], alpha=0.2, color='green', label='Λ-like')
ax.axhline(-1, color='gray', ls='--', lw=1)
ax.set_xlabel('INFLUX_PEAK')
ax.set_ylabel('w_eff')
ax.set_title('Influx Peak Sensitivity')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Entropy Sensitivity
ax = axs[0, 1]
ax.errorbar(df_entropy_k['param_value'], df_entropy_k['final_w_eff'], 
            yerr=df_entropy_k['w_eff_std'], fmt='o-', capsize=3, color='C1')
ax.axhspan(lambda_band[0], lambda_band[1], alpha=0.2, color='green')
ax.axhline(-1, color='gray', ls='--', lw=1)
ax.set_xlabel('ENTROPY_SENSITIVITY_K')
ax.set_ylabel('w_eff')
ax.set_title('Entropy Sensitivity')
ax.grid(True, alpha=0.3)

# 3. Imbalance Coupling
ax = axs[0, 2]
ax.errorbar(df_coupling['param_value'], df_coupling['final_w_eff'], 
            yerr=df_coupling['w_eff_std'], fmt='o-', capsize=3, color='C2')
ax.axhspan(lambda_band[0], lambda_band[1], alpha=0.2, color='green')
ax.axhline(-1, color='gray', ls='--', lw=1)
ax.set_xlabel('IMBALANCE_COUPLING')
ax.set_ylabel('w_eff')
ax.set_title('Coupling Strength')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

# 4. BH Radius
ax = axs[1, 0]
ax.errorbar(df_bh_radius['param_value'], df_bh_radius['final_w_eff'], 
            yerr=df_bh_radius['w_eff_std'], fmt='o-', capsize=3, color='C3')
ax.axhspan(lambda_band[0], lambda_band[1], alpha=0.2, color='green')
ax.axhline(-1, color='gray', ls='--', lw=1)
ax.set_xlabel('BH_ABSORB_RADIUS')
ax.set_ylabel('w_eff')
ax.set_title('BH Absorption Radius')
ax.grid(True, alpha=0.3)

# 5. Dilution Power
ax = axs[1, 1]
ax.errorbar(df_dilution['param_value'], df_dilution['final_w_eff'], 
            yerr=df_dilution['w_eff_std'], fmt='o-', capsize=3, color='C4')
ax.axhspan(lambda_band[0], lambda_band[1], alpha=0.2, color='green')
ax.axhline(-1, color='gray', ls='--', lw=1)
ax.set_xlabel('DILUTION_POWER')
ax.set_ylabel('w_eff')
ax.set_title('Dilution Exponent')
ax.grid(True, alpha=0.3)

# 6. Summary histogram
ax = axs[1, 2]
all_w_eff = np.concatenate([
    df_influx_peak['final_w_eff'].values,
    df_entropy_k['final_w_eff'].values,
    df_coupling['final_w_eff'].values,
    df_bh_radius['final_w_eff'].values,
    df_dilution['final_w_eff'].values
])
ax.hist(all_w_eff, bins=20, edgecolor='black', alpha=0.7)
ax.axvline(-1, color='red', ls='--', lw=2, label='w = -1 (Λ)')
ax.set_xlabel('w_eff')
ax.set_ylabel('Count')
ax.set_title('Distribution of w_eff Across All Sweeps')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('parameter_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to: parameter_sensitivity.png")

## Summary Statistics

In [ ]:
print("="*60)
print("PARAMETER SENSITIVITY SUMMARY")
print("="*60)

# Count runs in Λ-like region
n_total = len(all_w_eff)
n_lambda_like = np.sum((all_w_eff > -1.5) & (all_w_eff < -0.7))
n_cosmological_constant = np.sum((all_w_eff > -1.1) & (all_w_eff < -0.9))

print(f"\nTotal runs: {n_total}")
print(f"Λ-like (-1.5 < w < -0.7): {n_lambda_like} ({100*n_lambda_like/n_total:.1f}%)")
print(f"Cosmological constant-like (-1.1 < w < -0.9): {n_cosmological_constant} ({100*n_cosmological_constant/n_total:.1f}%)")

print(f"\nw_eff range: [{all_w_eff.min():.3f}, {all_w_eff.max():.3f}]")
print(f"w_eff mean ± std: {all_w_eff.mean():.3f} ± {all_w_eff.std():.3f}")

print("\nPer-Parameter Ranges:")
print(f"  INFLUX_PEAK:      w_eff in [{df_influx_peak['final_w_eff'].min():.3f}, {df_influx_peak['final_w_eff'].max():.3f}]")
print(f"  ENTROPY_K:        w_eff in [{df_entropy_k['final_w_eff'].min():.3f}, {df_entropy_k['final_w_eff'].max():.3f}]")
print(f"  COUPLING:         w_eff in [{df_coupling['final_w_eff'].min():.3f}, {df_coupling['final_w_eff'].max():.3f}]")
print(f"  BH_RADIUS:        w_eff in [{df_bh_radius['final_w_eff'].min():.3f}, {df_bh_radius['final_w_eff'].max():.3f}]")
print(f"  DILUTION_POWER:   w_eff in [{df_dilution['final_w_eff'].min():.3f}, {df_dilution['final_w_eff'].max():.3f}]")

print("\n" + "="*60)

## Interpretation

**Fine-tuning assessment:**
- If >50% of runs fall in the Λ-like region across broad parameter ranges → **not fine-tuned**
- If w_eff is highly sensitive to one parameter → that parameter is **critical**
- If w_eff is stable across most parameters → the mechanism is **robust**

**Next steps:**
1. Identify which parameters most strongly affect w_eff
2. Run 2D sweeps for critical parameter pairs
3. Compare stability regions with physical constraints